# EE 596 Mini Project
Weerasingha W.K.H.M (E/18/383)

In [97]:
# Import the libraries
import cv2
import numpy as np 
import matplotlib.pyplot as plt
import os
import huffman
import shutil

# Encode the Video

Read the video

In [98]:
# Organize the directory paths
path = os.getcwd()
Original_Frames_path = os.path.join(path, "Original_Frames")
Residual_Frames_path = os.path.join(path, "Residual_Frames")
Encoded_files_path = os.path.join(path, "Encoded_files")
Decode_frames_path = os.path.join(path, "Decode_frames")

In [99]:
# Establish the file paths
try:
    os.mkdir(Original_Frames_path)
    os.mkdir(Residual_Frames_path)
    os.mkdir(Encoded_files_path)
    os.mkdir(Decode_frames_path)
except:
    shutil.rmtree(Original_Frames_path)
    shutil.rmtree(Residual_Frames_path)
    shutil.rmtree(Encoded_files_path)
    shutil.rmtree(Decode_frames_path)
    os.mkdir(Original_Frames_path)
    os.mkdir(Residual_Frames_path)
    os.mkdir(Encoded_files_path)
    os.mkdir(Decode_frames_path)


In [100]:
# Store the frames and give limitaion for frame count
video_frames = []
frame_Count = 10

In [101]:
# Get  frames one by one from the video
vidcap = cv2.VideoCapture('Original_Image.mp4')
fps = vidcap.get(cv2.CAP_PROP_FPS)
success,image = vidcap.read()
count = 0
# while success:
while count<frame_Count:
  cv2.imwrite(os.path.join(Original_Frames_path, "frame%d.jpg" % count), image)     # save colured frames as JPEG file
  video_frames.append(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)) # Convert to gray image
  success,image = vidcap.read()
  count += 1

In [102]:
# User defined for user purpose
# frame_Count = count

In [103]:
# Print necessary details of frames
print("Frame counts : ", len(video_frames))

Frame counts :  10


Resize the frames to fit 8x8 macroblocks

In [104]:
# Get the height and width of the original image
height, width = video_frames[0].shape

# calculate the counts of the 8x8 macroblocks
block_size = 8
height_blocks = height // block_size
width_blocks = width // block_size

# Calculate new image dimensions
new_height = height_blocks * 8
new_width = width_blocks * 8


In [105]:
# Updated the frame array using resized frames
count = 0
for frame in video_frames:
    video_frames[count] = cv2.resize(frame, (new_width, new_height))
    count+=1

# Find residuals

In [106]:
# Function to calculate the sum of squared differences between two blocks
def block_matching_error(block1, block2):
    # return np.sum((block1 - block2) ** 2)
    return np.sum(np.abs(block1 - block2))

In [107]:
# Function for Logarithmic Search motion estimation
def logarithmic_search(i_frame, p_frame, block_size):
    height, width = i_frame.shape[:2]
    
    # Initialize matrix to store motion vectors
    motion_vectors = np.zeros((height // block_size, width // block_size, 2), dtype=np.int32)
   
    # Loop through each block in the target frame
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            target_block = p_frame[y:y+block_size, x:x+block_size]
            best_match_error = float('inf')
            best_match_vector = (0, 0)
            
            # Iterate through candidate motion vectors in a logarithmic pattern
            for dy in [-3, 0, 3]:
                for dx in [-3, 0, 3]:
                    ny, nx = y + dy, x + dx
                    if 0 <= ny < height - block_size + 1 and 0 <= nx < width - block_size + 1:
                        reference_block = i_frame[ny:ny+block_size, nx:nx+block_size]
                        error = block_matching_error(target_block, reference_block)
                        
                        # Update if the current candidate is a better match
                        if error < best_match_error:
                            best_match_error = error
                            best_match_vector = (dy, dx)
            
            # Update the motion vector matrix
            motion_vectors[y // block_size, x // block_size] = best_match_vector
    
    return motion_vectors

In [108]:
# Function to calculate the residual image
def calculate_residual(i_frame, p_frame, motion_vectors, block_size):
    height, width = i_frame.shape[:2]
    residual_image = np.zeros_like(i_frame)

    # Loop through each block in the frames
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            dy, dx = motion_vectors[y // block_size, x // block_size]
            target_block = p_frame[y:y+block_size, x:x+block_size]
            predicted_block = i_frame[y+dy:y+dy+block_size, x+dx:x+dx+block_size]
            
            # Calculate the difference between the target and predicted block
            residual_image[y:y+block_size, x:x+block_size] = target_block - predicted_block
    
    return residual_image

In [109]:
# List for saves
motion_vectors = []
residual_frames = []

In [110]:
# Find residual for each p frames
count = 0
while(count<len(video_frames)):
    # print("step 1 ", count)
    residual_frames.append(video_frames[count])
    for i in range(10-1):
        if (i+count+1==len(video_frames)):
            count = len(video_frames)
            break
        else:
            i_frame = video_frames[i+count]
            # print("step 2 ", count, i+count)
            p_frame = video_frames[i+count+1]
            
            block_size = 8
            motion_vec = logarithmic_search(i_frame, p_frame, block_size)
            motion_vectors.append(motion_vec)

            # Calculate the residual image
            residual_frames.append(calculate_residual(i_frame, p_frame, motion_vec, block_size))

    count+=10

Save the residual images

In [111]:
# Save the I and Residual frames
count = 0
for resi_img in residual_frames:
    if int(count%10) == 0:
        cv2.imwrite(os.path.join(Residual_Frames_path, "%d_I_frame.jpg" % count), resi_img)
    else:
        cv2.imwrite(os.path.join(Residual_Frames_path, "%d_residual.jpg" % count), resi_img)
    count+=1

# Encoding part

In [112]:
# Define Quntization matrix

ref_matrix = np.array([[16,  11,  10,  16,  24,  40,  51,  61],
                                [12,  12,  14,  19,  26,  58,  60,  55],
                                [14,  13,  16,  24,  40,  57,  69,  56],
                                [14,  17,  22,  29,  51,  87,  80,  62],
                                [18,  22,  37,  56,  68, 109, 103,  77],
                                [24,  35,  55,  64,  81, 104, 113,  92],
                                [49,  64, 78,  87, 103, 121, 120, 101],
                                [72,  92,  95,  98, 112, 100, 103 , 99]])  

# Low Quality 
quantization_matrix_low = np.round(ref_matrix*2).astype(int)

# Medium Quality 
quantization_matrix_medium = ref_matrix 

# High Quality
quantization_matrix_high = np.round(ref_matrix/2).astype(int)

YesorNo = int(input("Do you want to fit a bitrate (Yes : 1, No : 0): "))

if YesorNo == 1 :
    try:
        bitrate = int(input("Enter the bitrate(kbps) or press enter to set predefined bitrate(683kbps) : "))*1000
    except:
        bitrate = 683000 # Get the bitrate value
    quality = 1 # Set to default quantization level
else:
    # Ascked the quality level which want 
    bitrate = "Not Specified"
    quality = int(input("Select the quality level (low:0, Medium:1, High:2) : "))

# Assign the values for variables according to the input level
if quality == 0:
    quantization_matrix = quantization_matrix_low 
    output_file_path = "encoded_low_quality.txt"
    image_output =  "Quantized_low_quality.jpg"
    huffman_path = "Huffman_low_quality.txt"

elif quality == 1:
    if YesorNo == 1:
        quantization_matrix = quantization_matrix_medium
        output_file_path = "encoded_custon_bitrate%d.txt"%bitrate
        image_output =  "Quantized_custom_bitrate_%d.jpg"%bitrate
        huffman_path = "Huffman_medium_quality%d.txt"%bitrate
    else:
        quantization_matrix = quantization_matrix_medium
        output_file_path = "encoded_medium_quality.txt"
        image_output =  "Quantized_medium_quality.jpg"
        huffman_path = "Huffman_medium_quality.txt"

else:
    quantization_matrix = quantization_matrix_high
    output_file_path = "encoded_high_quality.txt"
    image_output =  "Quantized_high_quality.jpg"
    huffman_path = "Huffman_high_quality.txt"

Functions

In [113]:
# Function to devide each frame in to marcoblocks
def macroblock(height_blocks, width_blocks, block_size, resize_img):
    # Get the macroblocks
    # create an empty 4D array to store the 8x8 marcoblocks
    macroblocks = np.zeros((height_blocks, width_blocks, block_size, block_size), dtype=np.uint8)

    # iterate over the height and width of the image and store the 8x8 blocks in the array
    for i in range(height_blocks):
        for j in range(width_blocks):
            macroblocks[i, j] = resize_img[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
    return macroblocks

In [114]:
# Function to apply dicrete cosine transform
def dct_macroblocks(macroblocks):
    # Create an empty array to store the DCT of each macroblock
    dct_macroblocks = np.zeros(macroblocks.shape, dtype=np.float32)

    # Apply DCT to each macroblock
    for i in range(macroblocks.shape[0]):
        for j in range(macroblocks.shape[1]):
            dct_macroblocks[i, j] = cv2.dct(macroblocks[i, j].astype(np.float32))

    return dct_macroblocks


In [115]:
# Function for quantize 
def qunatizer(matix, frame):
    # Create architectural metrix to store the results
    archi_matrix = np.zeros((frame.shape[0], frame.shape[1], frame.shape[2], frame.shape[3]))
    # Quantize each marcoblocks
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            # Quantize the image by rounding the pixel values to the nearest quantization level
            archi_matrix[i, j] = np.round(frame[i, j] / matix)
    
    return archi_matrix

In [116]:
# Function to recreate the 2D image matrix
def recreate(quantized_image):
    for i in  range((quantized_image.shape[0])):
            a = quantized_image[i, 0] 
            for j in range((quantized_image.shape[1])-1):
                    b = quantized_image[i, j+1] 
                    c = np.concatenate((a,b), axis=1) # Concatanate marcoblocks along rows
                    a = c
            if i == 0:
                    f = c
            if i>0:
                    Matrix_2D = np.concatenate((f,c), axis=0) # Concatanate marcoblocks along columns
                    f = Matrix_2D
    return Matrix_2D

In [117]:
# Function to find prbability distribution
def probability(Matrix_2D):
    symbols = {}
    for pixel in Matrix_2D.flatten():
        if (pixel) in symbols:
            symbols[(pixel)] += 1
        else:
            symbols[(pixel)] = 1

    # Calculate probability distribution
    total_pixels = Matrix_2D.shape[0]*Matrix_2D.shape[1]
    probability_distribution = {}
    for item in symbols:
        probability_distribution[item] = symbols[item]/total_pixels

    # Get the above result as a list
    return  [(str(symbol), weight) for symbol, weight in probability_distribution.items()]

In [118]:
# Function for compressing the image using huffman
def encoder(image, huffman):
    img_copy = image
    symbol =  huffman ## Halfman values
    codeWord = str()
    # Quantize the image
    for i in range(img_copy.shape[0]):
        for j in range(img_copy.shape[1]):
            # print(str(img_copy[i, j]))
            codeWord += symbol[(img_copy[i, j])]

    return  codeWord

Call the functions for encode

In [119]:
# List to save huffman mappings
huffman_codes = []
iteration_matrix = []

In [120]:
def operator( quantization_matrix, dct_macroblocks):
    # Quantized the image
    # quantized_image = qunatizer(macroblocks.shape[0], macroblocks.shape[1], quantization_matrix, dct_macroblocks)
    quantized_image = qunatizer(quantization_matrix, dct_macroblocks)
    Matrix_2D = recreate(quantized_image)
    sym_weight_lst = probability(Matrix_2D)

    # Create a dictionary for the huffman codebook
    huff_map = huffman.codebook(sym_weight_lst)
    huff_map = dict([(float(symbol), weight) for symbol, weight in huff_map.items()])

    encoded_original = (encoder(Matrix_2D, huff_map))

    return encoded_original, huff_map, Matrix_2D

In [121]:
H, W = video_frames[0].shape
factor = H*W

In [122]:
# Encoding part
count = 0
for frame in residual_frames:

    # Devide each frame into marcoblocks
    macroblocks = macroblock(height_blocks, width_blocks, block_size, frame)

    # Apply DCT to each macroblock
    dct_image = dct_macroblocks(macroblocks)

    encoded_original, huff_map, Matrix_2D = operator( quantization_matrix, dct_image)
    print(len(encoded_original))
    
    # Set a variable for future purpose
    bot = encoded_original

    # Fit to bitrate 

    # This function is used for fit the image to a specific bitrate.

    if YesorNo == 1:
        # When speecific bitrate is lower than, after encode using medium quality quantization matrix
        if len(bot) > bitrate:
            iterate = 1.5
            print("bitrate lower")
            # Itterate the while loop to fit specified bitrate
            while (len(encoded_original)>bitrate):
                # store the previous parameters
                previous_encode = encoded_original; previous_huff = huff_map; previous_2D = Matrix_2D

                encoded_original, huff_map, Matrix_2D = operator( quantization_matrix*iterate, dct_image)

                # Find the compression ration
                CR_value = (factor*8)/len(encoded_original)

                # Check the condition for compression ration
                if (CR_value>=9):
                    encoded_original = previous_encode; huff_map = previous_huff; Matrix_2D = previous_2D
                    print("Couldn't fit to your bitrate. Last iterate CR = ", CR_value)
                    # print("Set to this CR = ", (factor*8)/len(previous_encode))
                    # print("Set to this bitrate = ", len(previous_encode))
                    iterate = iterate + 0.5
                    break
                
                iterate = iterate + 0.5
                print("Next iterate bitrate : ",len(encoded_original), ", Q-matrix Multiply value: ", (iterate-0.5))

            iterate = iterate-0.5
                
        # When speecific bitrate is higher than, after encode using medium quality quantization matrix
        else:
            iterate = 2
            print("bitrate higher")
            # Itterate the while loop to fit specified bitrate
            while (len(encoded_original)<bitrate):
                # store the previous parameters
                gost = encoded_original; gost_huff = huff_map; gost_2D = Matrix_2D

                encoded_original, huff_map, Matrix_2D = operator( quantization_matrix/iterate, dct_image)

                # Find the compression ration
                CR_value = (factor*8)/len(encoded_original)

                # Check the condition for compression ration
                if (CR_value<=3):
                    print("Couldn't fit to your bitrate. Last iterate CR = ", CR_value)
                    # print("Set to this CR = ", (factor*8)/len(gost))
                    # print("set to this bitrate = ", len(gost))
                    iterate = iterate + 1
                    break

                iterate = iterate + 1
                print("Next iterate bitrate : ", len(encoded_original), ", Q-matrix Multiply value: ", 1/(iterate-1))

            encoded_original = gost; huff_map = gost_huff; Matrix_2D = gost_2D; iterate = 1/(iterate-2)

        # Get the quantization matrix multiply factor 
        iteration_matrix.append(iterate)
        print("Finaly, fitted bitrate = ", len(encoded_original), ", Multyplier = ", iterate)
        print("CR = ", (factor*8)/len(encoded_original))
    else:
        iteration_matrix.append(1)

    huffman_codes.append(huff_map)

    #Save the encoded data to a text file
    file =  open(os.path.join(Encoded_files_path, "encode_frame_%d.txt" % count), 'w')

    # Encode the each marcoblock separately and save result in the text file
    file.write(str(encoded_original))

    file.close()
    
    count+=1
    

682801
bitrate lower
Next iterate bitrate :  647464 , Q-matrix Multiply value:  1.5
Next iterate bitrate :  626487 , Q-matrix Multiply value:  2.0
Next iterate bitrate :  612171 , Q-matrix Multiply value:  2.5
Next iterate bitrate :  602088 , Q-matrix Multiply value:  3.0
Next iterate bitrate :  594086 , Q-matrix Multiply value:  3.5
Next iterate bitrate :  587430 , Q-matrix Multiply value:  4.0
Next iterate bitrate :  581866 , Q-matrix Multiply value:  4.5
Next iterate bitrate :  577405 , Q-matrix Multiply value:  5.0
Next iterate bitrate :  573541 , Q-matrix Multiply value:  5.5
Next iterate bitrate :  570069 , Q-matrix Multiply value:  6.0
Next iterate bitrate :  567205 , Q-matrix Multiply value:  6.5
Next iterate bitrate :  564756 , Q-matrix Multiply value:  7.0
Next iterate bitrate :  562436 , Q-matrix Multiply value:  7.5
Next iterate bitrate :  560332 , Q-matrix Multiply value:  8.0
Next iterate bitrate :  558507 , Q-matrix Multiply value:  8.5
Next iterate bitrate :  556780 , Q

# Decoding part

In [123]:
# Function to the decode the compressed haufman coded file
def decoder(data, size, txtfile):
    codeVal = []
    data_new = dict([(value, key) for key, value in data.items()])

    code_list = list(data.values())

    i = 0
    # decode the read file's data
    while i != len(txtfile):
        # Check with the huffman codes
        for j in range(len(code_list)):
            a0 = txtfile[i:(i+len(code_list[j]))]
            if (a0 == code_list[j]):
                codeVal.append(data_new[code_list[j]])
                i+=len(code_list[j])
                break

    
    merge_image = np.array(codeVal).reshape(size)
    merge_image = merge_image.astype(float)

    return merge_image

In [124]:
# List to save huffman decoded matrix and decoded frames bitrates
huffman_encoded_frames = []
decode_bitrate = []

In [125]:
# Decode the encoded image
for count in range(len(video_frames)):
    # Read the saved file line by line
    file1 = open(os.path.join(Encoded_files_path, "encode_frame_%d.txt"%count), 'r')
    Lines = file1.read()
    decode_bitrate.append(len(Lines))

    if len(Lines)==0:
        huffman_encoded_frames.append(np.zeros((video_frames[0].shape)))
    else:
        # Encode the text file and create the 4D matrix 
        huffman_encoded_frames.append(decoder(huffman_codes[count], video_frames[0].shape, Lines))
                
    file1.close()
    

In [126]:
# Fuunction for Dequantize 

def dequantized(height_blocks, width_blocks, block_size, frame, quanti_matrix):
    # Create a 4D matrix to store dequantized result
    decoded_quantized = np.zeros((height_blocks, width_blocks, block_size, block_size))

    # Dequantize occur marcoblock by marcoblock
    for i in range(frame.shape[0]):
            for j in range(frame.shape[1]):
                decoded_quantized[i, j] = (frame[i,j] * quanti_matrix)
    return decoded_quantized

In [127]:
# Function for IDCT
def IDCT(frame_macro):
    # Create an empty array to store the IDCT of each macroblock
    inverse_dct = np.zeros(frame_macro.shape)

    # Apply DCT to each macroblock
    for i in range(frame_macro.shape[0]):
        for j in range(frame_macro.shape[1]):
            inverse_dct[i, j] = (cv2.idct(np.float32(frame_macro[i, j])))
    
    return inverse_dct

In [128]:
# List to save reconstructed residuals
reconstruct_residuals = []

In [129]:
# reconstruct
for count in range(len(huffman_encoded_frames)):
    # Re construct the marcoblocks

    # create an empty 4D array to store the 8x8 marcoblocks
    macroblocks_2 = np.zeros((height_blocks, width_blocks, block_size, block_size)).astype(float)

    # iterate over the height and width of the image and store the 8x8 blocks in the array
    for i in range(height_blocks):
        for j in range(width_blocks):
            macroblocks_2[i, j] = huffman_encoded_frames[count][i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]


    I_macroblocks = macroblocks_2
    
    I_dequantized = dequantized(height_blocks, width_blocks, block_size, I_macroblocks, quantization_matrix*iteration_matrix[count])
    
    I_idct = IDCT(I_dequantized)

    I_receate = recreate(I_idct)

    # Clip the pixel values to the valid range (0-255)
    I_reconstruct = np.clip(I_receate, 0, 255).astype(np.uint8)
    
    reconstruct_residuals.append(I_reconstruct)


In [130]:
# plt.imshow(reconstruct_residuals[0], cmap = 'gray')

Predict P frames

In [131]:
# Function to predict P-frame
def get_residual( residual_image, block_size, height, width, i_frame, motion_vector_array):
    predicted_frame = np.zeros_like(residual_image)
    # Loop through each block and predict the second frame
    for y in range(0, height, block_size):
        for x in range(0, width, block_size):
            dy, dx = motion_vector_array[y // block_size, x // block_size]
            predicted_frame[y:y+block_size, x:x+block_size] = i_frame[y+dy:y+dy+block_size, x+dx:x+dx+block_size] + residual_image[y:y+block_size, x:x+block_size]
    
    return predicted_frame


In [132]:
# List save predicted images
predicted_frames = []

In [133]:
# Predict the second frame using I frame, motion vectors, and residual image
height, width = reconstruct_residuals[0].shape[:2]

count = 0
m_count = 1
for frame in reconstruct_residuals:
    if (count%10) == 0:
        predicted_frames.append(reconstruct_residuals[count])
    else:
        get = get_residual(reconstruct_residuals[count], block_size, height, width, predicted_frames[count-1], motion_vectors[m_count-1])
        predicted_frames.append(get)
        m_count+=1
        
    count+=1


Function to find PSNR values

In [134]:
# Function to find PSNR
def PSNR(image1, image2):
    mse = np.mean((np.subtract(image1, image2)**2))
    max_pixel = 255
    psnr = 20 * np.log10(max_pixel) - 10 * np.log10(mse)
    return psnr

In [135]:
# File path to save PSNR and CR
psnr_cr_file = "PSNR_CR.txt"

In [136]:
# Save the predicted frames and find the PSNR values using grascaled original images and decoded images
count = 0
H, W = video_frames[0].shape

file =  open(psnr_cr_file, 'w')
try:
    word = "Input Bitrate(kbps) = "+str(bitrate/1000)+"\n"
except:
    word = "Input Bitrate(kbps) = "+str(bitrate)+"\n"
file.write(word)
for pred_img in predicted_frames:
    if (count%10) == 0:
        cv2.imwrite(os.path.join(Decode_frames_path, "%d_Decode_Predic_I_frame.jpg" % count), pred_img)
    else:
        cv2.imwrite(os.path.join(Decode_frames_path, "%d_Decode_Predict_P_frame.jpg" % count), pred_img)
        
    # print("PSNR Value of frame %d = "%count, PSNR(video_frames[count], np.clip(pred_img, 0, 255).astype(np.uint8)), end=' ')
    # print(", CR = ", (H*W*8)/(decode_bitrate[count]))
    a = "PSNR Value of frame %d = "%count + str(PSNR(video_frames[count], np.clip(pred_img, 0, 255).astype(np.uint8))) + ", CR = " + str((H*W*8)/(decode_bitrate[count])) +"\n"
    file.write(a)
    count+=1

file.close()

In [137]:
# plt.imshow(predicted_frames[9], cmap = 'gray')

_________________________________

# Recreate the video

In [138]:
# Video output file path
video_name = 'output_video.mp4'

In [139]:
# Step 2: Create a video using the images in the list
w, h = predicted_frames[0].shape
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'H264'), fps, (h, w))

for img in predicted_frames:
    
    video.write(img)
    
cv2.destroyAllWindows()
video.release()
